# Cloud Computing Group-4


#### Read Function definition

In [1]:
import networkx.algorithms.approximation
import networkx as nx

def read_network(filename = 'as19971108.txt'):
    G = nx.Graph()
    i =  0
    print('Reading file {filename}'.format(filename=filename))
    with open(filename, 'r') as f:
        for line in f.readlines()[4:]:
            edge = line.replace("\n","").split("\t")
            if edge[0] != edge[1]:
                G.add_edge(edge[0],edge[1],weight = random.random())
    print('New network generated')
    return G
    

#### Random Tree Generator

In [2]:
import random

def get_weight(edge, network):
    weight = network[edge[0]][edge[1]]['weight']
    return weight

def validate_graph(graph, network):
    if len(graph.nodes()) != len(network.nodes()):
        return False
    for edge in graph.edges:
        if edge not in network.edges:
            return False
    return nx.is_connected(graph)

# Tree validation function
def validate_tree(tree, network):
    if not validate_graph(tree, network):
        return False
    if len(nx.nx.cycle_basis(tree)) > 0:
        return False
    return True

def random_spanning_tree_generator(network, n_trees=20):
    print('generating {n_trees} random spanning trees'.format(n_trees=n_trees))
    nodes = network.nodes()
    trees = []
    while(len(trees) < n_trees):
        tree = nx.Graph()
        next_nodes = [random.choice(list(nodes))]
        while len(next_nodes)>0:
            edges = network.edges(next_nodes.pop())
            edges = list(edges)
            random.shuffle(edges)
            for edge in edges:
                if edge[1] not in tree.nodes():
                    tree.add_edge(edge[0],edge[1],weight=get_weight(edge, network))
                    next_nodes.append(edge[1])
        if validate_tree(tree, network):
            trees.append(tree)
    print('Done!')
    return trees
                
        
        

#### Fitness Function

In [3]:
def fitness_function(G):
    cost = 0
    for edge in G.edges():
        cost += get_weight(edge, G)
    for node in list(G.nodes()):
        cost  += G.degree(node)
    return cost

#### Chromosome graph representation

In [4]:
import numpy as np
def get_chromosome(graph, network_edges):
    return [edge in graph.edges for edge in network_edges]

def get_graph(chromosome, network_edges, network):
    G = nx.Graph()
    network_edges_list = np.array(list(network_edges))
    edges = network_edges_list[chromosome]
    for edge in edges:
        G.add_edge(edge[0], edge[1], weight=get_weight(edge, network))
    return G
    

#### Get Child function

In [5]:
def crossover(gen1, gen2, network_edges, network):
    child_1 = gen1.copy()
    child_2 = gen1.copy()
    gen_crossover_sample = random.sample(range(len(gen1)), int(len(gen1)*0.5))
    for i in range(len(gen1)):
        if i in gen_crossover_sample:
            aux = child_1[i]
            child_1[i] = child_2[i]
            child_2[i] = aux
    if fitness_function(get_graph(child_1, network_edges, network)) > fitness_function(get_graph(child_1, network_edges, network)):
        return child_1
    return child_2

def mutation(gen):
    child = gen.copy()
    gen_crossover_sample = random.sample(range(len(gen)), int(len(gen)*0.05))
    for i in range(len(gen)):
        if i in gen_crossover_sample:
            child[i] = not child[i]
    return child

iterator = 0

def get_child(parents, network_edges, network):
    global iterator
    child_found = False
    while (not child_found):
        if iterator % 11 == 0:
            parent = get_chromosome(random.choice(parents), network_edges)
            child = get_graph(mutation(parent), network_edges, network)
            if validate_graph(child,network):
                child_found = True
        else:
            parent1 = get_chromosome(random.choice(parents), network_edges)
            parent2 = get_chromosome(random.choice(parents), network_edges)
            child = get_graph(crossover(parent1, parent2, network_edges, network), network_edges, network)
            if validate_graph(child,network):
                child_found = True
        iterator +=1
    return child
        
        

### GA function

In [6]:
def GA(population, network_edges, network, n_iter = 10, max_tries = 1000):
    print('GA algorithm started')
    tries = 0
    iter_no =0
    n_childs = len(population)
    parents = population.copy()
    min_fitness = 100000000
    while(iter_no < n_iter):
        for child in parents:
            new_fitness = fitness_function(child)
            if new_fitness < min_fitness:
                min_fitness = new_fitness
                best_graph = child
        children = []
        while(len(children) < n_childs):
            child = get_child(parents, network_edges, network)
            if fitness_function(child) < min_fitness:
                children.append(child)
            tries +=1
            if tries > max_tries:
                return best_graph
        
        parents = children.copy()
        print('Finished offspring generation No.{iter_no}'.format(iter_no=iter_no))
    return best_graph
    

## Running GA

In [7]:
G = read_network()
network_edges = G.edges()
population = random_spanning_tree_generator(G, 20)
best_graph = GA(population, network_edges, G)
print('Best Graph Found!')

Reading file as19971108.txt
New network generated
generating 20 random spanning trees
Done!
GA algorithm started
Best Graph Found!


In [8]:
print(best_graph.edges())

[('2529', '1849'), ('2529', '1755'), ('2529', '3561'), ('2529', '174'), ('2529', '5459'), ('2529', '2516'), ('2529', '5417'), ('1849', '2876'), ('1849', '8383'), ('1849', '6894'), ('1755', '5405'), ('1755', '3208'), ('1755', '6708'), ('1755', '6707'), ('1755', '2561'), ('1755', '6741'), ('1755', '3278'), ('1755', '3233'), ('1755', '2607'), ('1755', '5379'), ('1755', '5629'), ('1755', '5435'), ('1755', '6666'), ('1755', '5424'), ('1755', '8434'), ('3561', '5794'), ('3561', '4184'), ('3561', '7435'), ('3561', '6258'), ('3561', '4670'), ('3561', '5603'), ('3561', '10460'), ('3561', '10680'), ('3561', '4798'), ('3561', '6646'), ('3561', '6549'), ('3561', '7265'), ('3561', '3458'), ('3561', '6348'), ('3561', '7234'), ('3561', '4272'), ('3561', '4293'), ('3561', '763'), ('3561', '6584'), ('3561', '8226'), ('3561', '7920'), ('3561', '268'), ('3561', '6163'), ('3561', '4279'), ('3561', '4286'), ('3561', '1908'), ('3561', '6536'), ('3561', '10478'), ('3561', '4574'), ('3561', '7726'), ('3561', 